# Chapter 6 - Looping & Iteration
___
## 6.1 Writing Pythonic Loops

## Summary 
* Writing C-style loops in Python is considered unpythonic.
Avoid managing loop indexes and stop conditions manually if
possible.
* Python’s for-loops are really “for-each” loops that can iterate
directly over items from a container or sequence.

In [1]:
# C or Java loops

my_items = ['a', 'b', 'c']

i = 0
while i < len(my_items):
    print(my_items[i])
    i += 1

a
b
c


In [2]:
# Java loops
for i in range(len(my_items)):
    print(my_items[i])

a
b
c


In [10]:
# for each python loop  
for item in my_items:
    print(item)

a
b
c


In [11]:
# enumerate 
for i, item in enumerate(my_items):
    print(f'{i} : {item}')

0 : a
1 : b
2 : c


In [12]:
# pythonic looping through dictionary 
emails = {
    'Bob' : 'bob@example.com',
    'Alice' : 'alice@example.com', 
}

for name, email in emails.items():
    print(f'{name} -> {email}')

Bob -> bob@example.com
Alice -> alice@example.com


In [13]:
# C-style loop 
# for (int i = a; i < n; i += s) {
#      ...
# }

for i in range(1, 10, 2):
    print(i)

1
3
5
7
9


___
## 6.2 Comprehending Comprehensions

*Synactic sugar* for looping in a pythonic manner. One level of
nesting for comprehensions is the limit. After that, in most cases it’s better (as in “more readable” and “easier to maintain”) to use for-loops beyond that point.

```Python
values = [expression for item in collection]

values = []
for item in collection:
    values.append(expression)
```

### Equivalence with conventional syntax

In [22]:
squares = [x*x for x in range(10)]
squares

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [23]:
squares = []

for x in range(10):
    squares.append(x*x)

squares

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

### conditional logic 

In [24]:
even_squares = [x*x for x in range(10) if x % 2 == 0]
even_squares

[0, 4, 16, 36, 64]

In [25]:
even_squares = []
for x in range(10):
    if x % 2 == 0:
        even_squares.append(x * x)
even_squares

[0, 4, 16, 36, 64]

### Set and dictionary comprehensions

It should be noted that sets are an unordered collection type. So you get **random** order when you add items to a ```set``` container

In [26]:
# sets are unordered
{x*x for x in range(-9,10)}

{0, 1, 4, 9, 16, 25, 36, 49, 64, 81}

In [27]:
# dictionary
{x: x*x for x in range(5)}

{0: 0, 1: 1, 2: 4, 3: 9, 4: 16}

___
## 6.3 List Slicing Tricks and the sushi operator 

```Python
lst[start:end:stop]
```
### Summary
* The ```:``` “sushi operator” is not only useful for selecting sublists
of elements within a list. It can also be used to clear, reverse,
and copy lists.
* But be careful—this functionality borders on the arcane for
many Python developers. Using it might make your code less
maintainable for everyone else on your team.

In [47]:
lst = [x for x in range(6)]
lst

[0, 1, 2, 3, 4, 5]

In [48]:
# start end and step
lst[1:3:1]

[1, 2]

In [49]:
# default step size is 1
lst[1:5]

[1, 2, 3, 4]

#### Sushi operator

In [50]:
# sushi operator
lst[::2]

[0, 2, 4]

In [51]:
# reversing the list order
lst[::-1]

[5, 4, 3, 2, 1, 0]

#### clearning contents

In [52]:
# deleting the contents of a list but not the object
lst = [1,2,3,4,5]
del lst[:]
lst

[]

In [53]:
# equivalent
lst = [1,2,3,4,5]
lst.clear()
lst

[]

#### replacing elements without deleting the object

In [54]:
# replace all elements without creating a new list object
original_lst = lst
lst[:] = [7, 8, 9]
lst

[7, 8, 9]

In [55]:
original_lst

[7, 8, 9]

In [56]:
original_lst is lst

True

#### Shallow copies

In [46]:
copied_lst = lst[:]
copied_lst is lst

False

___
## 6.4 Beautiful Iterators

### Summary 

* Iterators provide a sequence interface to Python objects that’s **memory efficient and considered Pythonic**. Behold the beauty
of the for-in loop!
* To support iteration an object needs to implement the iterator protocol by providing the ```__iter__``` and ```__next__``` dunder
methods.
* Class-based iterators are only one way to write iterable objects
in Python. Also consider generators and generator expressions

Python's elegant loop constructs work by making use of the **iterator protocol**. This protocol makes use of the:

* ```__iter__``` and ```__next__``` dunder methods in objects

### How do ```for-in``` loops work in python

```Python
# syntactic sugar
repeater = Repeater('Hello')
for item in repeater:
    print(item)

# equivalent
repeater = Repeater('Hello')
iterator = repeater.__iter__()
while True:
    item = iterator.__next__()
    print(item)       
```

### Pythonic for loops under the hood

In [84]:
class BoundedRepeater:
    def __init__(self, value, max_repeats):
        self.value = value
        self.max_repeats = max_repeats
        self.count = 0
    def __iter__(self):
        return self
    def __next__(self):
        if self.count >= self.max_repeats:
            raise StopIteration
        self.count += 1
        return self.value
    
    # Python 2 compatibility:
    def next(self):
        return self.__next__()

In [85]:
repeater = BoundedRepeater('Hello', 3)
for item in repeater:
    print(item)

Hello
Hello
Hello


In [87]:
# take away the syntactic sugar
repeater = BoundedRepeater('Hello', 3)
iterator = iter(repeater)
while True:
    try:
        item = next(iterator)
    except StopIteration:
        break
    print(item)

Hello
Hello
Hello


### Iterating forever

1. In the ```__init__``` method, we link each ```RepeaterIterator``` instance to the ```Repeater``` object that created it. That way we can
hold onto the “source” object that’s being iterated over.
2. In ```RepeaterIterator.__next__```, we reach back into the
“source” ```Repeater``` instance and return the value associated
with it.

If you’ve ever worked with database cursors, this mental model will
seem familiar: We first initialize the cursor and prepare it for reading,
and then we can fetch data from it into local variables as needed, one
element at a time.

In [69]:
class Repeater:
    def __init__(self, value):
        self.value = value
    # iter()
    def __iter__(self):
        return RepeaterIterator(self)
    
class RepeaterIterator:
    def __init__(self, source):
        self.source = source
    # next()
    def __next__(self):
        return self.source.value

In [70]:
repeater = Repeater('Hello')
i = 0 

while i < 10:
    print(next(iter(repeater)))
    i += 1

Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello


#### Equivalence between dunder methods and facade methods

In [76]:
repeater = Repeater('Hello')
iterator = repeater.__iter__()
iterator.__next__()

'Hello'

#### Simpler repeater class

In [77]:
class Repeater:
    def __init__(self, value):
        self.value = value
    def __iter__(self):
        return self
    def __next__(self):
        return self.value

In [78]:
repeater = Repeater('Hello')
repeater.__next__()

'Hello'

### Iterating with termination conditions

Iterators use **exceptions to structure control flow**. To signal the end of iteration, an iterator raises a ```StopIteration``` exception

In [80]:
my_list = [1,2,3]
iterator = iter(my_list)
next(iterator)
next(iterator)
next(iterator)

3

In [81]:
next(iterator)

StopIteration: 

___
## 6.5 Generators are simplified iterators

Writing class-based iterators requires a lot of boilerplate code. There is a more efficient way using ```generators``` and the ```yield``` keyword.

You see, when a ```return``` statement is invoked inside a function, it permanently passes control back to the caller of the function. When a ```yield``` is invoked, it also passes control back to the caller of the function—but it only does so temporarily.

Whereas a ```return``` statement disposes of a function’s local state, a
```yield``` statement **suspends the function and retains its local state**. In practical terms, this means **local variables and the execution state** of
the generator function are only **stashed away temporarily** and not thrown out completely. 

### Summary
* Generator functions are syntactic sugar for writing objects that
support the iterator protocol. Generators abstract away much
of the boilerplate code needed when writing class-based iterators.
* The ```yield``` statement allows you to temporarily suspend execution of a generator function and to pass back values from it.
* Generators start raising ```StopIteration``` exceptions after control flow leaves the generator function by **any means other than
a yield statement.**

In [2]:
# classed based approach
class Repeater:
    def __init__(self, value):
        self.value = value
    def __iter__(self):
        return self
    def __next__(self):
        return self.value

In [5]:
# pythonic way
def repeater(value):
    while True:
        yield value

In [7]:
re = repeater('hi')
next(re)

'hi'

Calling a generator function doesn't even run the function. It merely creates and returns a **generator object**. The code in the function only exectues when ```next()``` is called on the object.

In [8]:
repeater('Hey')

<generator object repeater at 0x0000023D5EB3F270>

In [9]:
generator_obj = repeater('Hey')
next(generator_obj)

'Hey'

### generators that stop generating 

In [13]:
def repeat_three_times(value):
    yield value 
    yield value 
    yield value

In [15]:
rtt = repeat_three_times('hi')
for _ in repeat_three_times('hi'):
    print(next(rtt))

hi
hi
hi


In [16]:
next(rtt)

StopIteration: 

### Bounded repeater with generators and yield

In [20]:
class BoundedRepeater:
    def __init__(self, value, max_repeats):
        self.value = value
        self.max_repeats = max_repeats
        self.count = 0
    def __iter__(self):
        return self
    def __next__(self):
        if self.count >= self.max_repeats:
            raise StopIteration
        self.count += 1
        return self.value
    
    # Python 2 compatibility:
    def next(self):
        return self.__next__()

In [21]:
# this is a much slimmed down version of the class based approach 
def bounded_repeater(value, max_repeats):
    count = 0
    while True:
        if count >= max_repeats:
            return
        count += 1
        yield value

In [22]:
for x in bounded_repeater('Hi', 4):
    print(x)

Hi
Hi
Hi
Hi


In [24]:
# 3 lines of code compared to 3
def bounded_repeater(value, max_repeats):
    for i in range(max_repeats):
        yield value

___
## 6.6 Generator expressions

Generator functions give you a shortcut for supporting the iterator protocol in your own code.Generator expressions give you an even more effective shortcut for writing iterators. With a simple and concise syntax that looks like a list comprehension, you’ll be able to define iterators in a single line of code.

```Python
genexpr = (expression for item in collection)

# equivalent code
def generator():
    for item in collection:
        yield expression
```

In [28]:
# 1 line of code vs 3
iterator = ('Hello' for i in range(3)) 

In [29]:
def bounded_repeater(value, max_repeats):
    for i in range(max_repeats):
        yield value


In [30]:
for i in iterator:
    print(i)

Hello
Hello
Hello


### list comprehensions vs generator expressions

In [31]:
listcomp = ['Hello' for i in range(3)]
genexpr = ('Hello' for i in range(3))

In [32]:
listcomp

['Hello', 'Hello', 'Hello']

In [33]:
genexpr

<generator object <genexpr> at 0x0000023D5F07BC10>

In [34]:
list(genexpr)

['Hello', 'Hello', 'Hello']

### filtering values 

```Python 
def generator():
    for item in collection:
        if condition:
            yield expression
```

In [36]:
even_squares = (x*x for x in range(10) if x % 2 == 0)

for x in even_squares:
    print(x)

0
4
16
36
64


### In-line Generator Expressions

You can create an iterator and consume it straight away

In [39]:
for x in ('Bom dia' for i in range(3)):
    print(x)

Bom dia
Bom dia
Bom dia


In [40]:
sum((x*2 for x in range(10)))

90

In [41]:
# if the generator is a single argument to a function we can drop the parenthesis
sum(x * 2 for x in range(10))

90

### Generalizing generator expressions
We can chain expressions and filtering clauses.
**Please don’t write deeply nested generator expressions like that. They
can be very difficult to maintain in the long run**

```Python 
(expr for x in xs if cond1
    for y in ys if cond2
    ...
    for z in zs if condN)

# equivalent
for x in xs:
    if cond1:
        for y in ys:
            if cond2:
                ...
                    for z in zs:
                        if condN:
                            yield expr

```

If you need to use nested generators and complex filtering conditions,
it’s usually better to factor out sub-generators (so you can name them)
and then to chain them together again at the top level.

___
## 6.7 Iterator Chains

### Summary
* Generators can be **chained together** to form highly efficient and
maintainable data processing pipelines.
* Chained generators **process each element going through the
chain individually**.
* **Generator expressions** can be used to write concise **pipeline definitions**, but this can **impact readability**.

In [56]:
def integers():
    for i in range(1,9):
        yield i

In [57]:
chain = integers()
list(chain)

[1, 2, 3, 4, 5, 6, 7, 8]

### Chaining Iterators

In [58]:
def squared(seq):
    for i in seq:
        yield i*i
        
def negated(seq):
    for i in seq:
        yield -i

In [59]:
chain = negated(squared(integers()))
list(chain)

[-1, -4, -9, -16, -25, -36, -49, -64]

The only downside to using generator expressions is that they can’t
be configured with function arguments, and you can’t reuse the same
generator expression multiple times in the same processing pipeline.

In [60]:
# equivalent expression
integers = range(8)
squared = (i * i for i in integers)
negated = (-i for i in squared)

In [61]:
negated

<generator object <genexpr> at 0x0000023D5EBDC970>

In [62]:
list(negated)

[0, -1, -4, -9, -16, -25, -36, -49]